<a href="https://colab.research.google.com/github/MGrgat1/deep_learning_in_google_colab/blob/main/lab4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example 1 - data loader

In [ ]:
# U ovom primjeru radit ćemo s MNIST setom podataka kako treniranje ne bi predugo trajalo
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time
from torchvision import datasets, transforms
from torch import nn, optim
from PIL import Image

transforms = torchvision.transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
# Skinemo podatke MNIST dataseta kao u četvrtom predavanju
trainset = datasets.MNIST('mnist_data', download=True, train=True, transform=transforms)
testset = datasets.MNIST('mnist_data', download=True, train=False, transform=transforms)

# Koristimo dataloader za uzimanje podataka iz setova podataka:
# Označavamo veličinu batch_size-a i treba li podatke izmješati (shuffle)
# Batch size jednostavno mjenjamo kasnije ukoliko je kod dobro napisan!
train_loader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
test_loader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)

dataiter = iter(train_loader)
images, labels = dataiter.next()

print(images.shape)
print(labels.shape)
# Ispisuje sljedeće:
# torch.Size([64, 1, 28, 28])
# torch.Size([64])
# Zašto?
# Zato što imamo 64 slikice u batch-u, slika ima jedan kanal (Greyscale) i velika je 28 x 28 piksela
# Postoji jedan label za jednu sliku (kategoriju) i imamo 64 slike i samim time 64 kategorije

#Prikažimo slike... Onako usput.
figure = plt.figure()
num_of_images = 60
for index in range(1, num_of_images + 1):
    plt.subplot(6, 10, index)
    plt.axis('off')
    plt.title(str(labels.numpy()[index]))
    plt.imshow(images[index].numpy().squeeze(), cmap='gray_r')
plt.show()




# Example 2 - NN GPU

In [ ]:
# U ovom primjeru radit ćemo s MNIST setom podataka kako treniranje ne bi predugo trajalo
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch import nn, optim
from PIL import Image

cuda = True if torch.cuda.is_available() else False
device = torch.device('cpu')
if cuda:
    device = torch.device('cuda')

transforms = torchvision.transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
# Skinemo podatke MNIST dataseta kao u četvrtom predavanju
trainset = datasets.MNIST('mnist_data', download=True, train=True, transform=transforms)
testset = datasets.MNIST('mnist_data', download=True, train=False, transform=transforms)

# Koristimo dataloader za uzimanje podataka iz setova podataka:
# Označavamo veličinu batch_size-a i treba li podatke izmješati (shuffle)
# Batch size jednostavno mjenjamo kasnije ukoliko je kod dobro napisan!
# pin_memory znači loading u GPU. Neće raditi ako nema više workera!
train_loader = DataLoader(trainset, batch_size=1024, shuffle=True, num_workers=32, pin_memory=True)
test_loader = DataLoader(testset, batch_size=1024, shuffle=True, num_workers=32, pin_memory=True)

# Kreiramo model:
# Ulaz je 784. Zašto? Zato što je 28 x 28 = 784
# Dakle, za svaki piksel imamo jedan ulaz u NN.
# Hidden veličine su nasumične
input_size = 784
hidden_size_1 = 128
hidden_size_2 = 64
output_size = 10

# Prvo imamo linearni sloj, pa aktivaciju, pa linearni sloj itd...
# Na kraju dodajemo softmax koji se standardno koristi prilikom klasifikacije.
model = nn.Sequential(nn.Linear(input_size, hidden_size_1),
                      nn.ReLU(),
                      nn.Linear(hidden_size_1, hidden_size_2),
                      nn.ReLU(),
                      nn.Linear(hidden_size_2, output_size),
                      nn.LogSoftmax(dim=1)).to(device)


loss_fn = nn.NLLLoss().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)




# Bilježimo početak
time0 = time()
# Idemo trenirati 15 epoha
epochs = 15
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:

        # Sjetimo se što radi ova komanda!
        images = images.to(device).view(images.shape[0], -1)

        # Nuliramo gradijente
        optimizer.zero_grad()

        # Tražimo izlaz iz mreže i računamo grešku
        output = model(images)
        loss = loss_fn(output, labels.to(device))

        # Propagiramo grešku unazad
        loss.backward()

        # Idemo na idući korak
        optimizer.step()

        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss / len(train_loader)))
print("\nTrenirali smo (u minutama):", (time() - time0) / 60)


# Example 3 - testing


In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch import nn, optim
from PIL import Image

cuda = True if torch.cuda.is_available() else False
device = torch.device('cpu')
if cuda:
    device = torch.device('cuda')

transforms = torchvision.transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

trainset = datasets.MNIST('mnist_data', download=True, train=True, transform=transforms)
testset = datasets.MNIST('mnist_data', download=True, train=False, transform=transforms)


train_loader = DataLoader(trainset, batch_size=64, shuffle=True, num_workers=32, pin_memory=True)
test_loader = DataLoader(testset, batch_size=64, shuffle=True, num_workers=32, pin_memory=True)


input_size = 784
hidden_size_1 = 128
hidden_size_2 = 64
output_size = 10

model = nn.Sequential(nn.Linear(input_size, hidden_size_1),
                      nn.ReLU(),
                      nn.Linear(hidden_size_1, hidden_size_2),
                      nn.ReLU(),
                      nn.Linear(hidden_size_2, output_size),
                      nn.LogSoftmax(dim=1)).to(device)


loss_fn = nn.NLLLoss().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

time0 = time()
epochs = 15

for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:

        images = images.to(device).view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model(images)
        loss = loss_fn(output, labels.to(device))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss / len(train_loader)))
print("\nTrenirali smo (u minutama):", (time() - time0) / 60)

# Poigrajte se samo s veličinom batch-a ;)
# Proučite kakav efekt ima veličina batch-size-a na treniranje!
# Krenite od: https://medium.com/mini-distill/effect-of-batch-size-on-training-dynamics-21c14f7a716e
correct_count, all_count = 0, 0
for images, labels in test_loader:
    for i in range(len(labels)):
        img = images[i].view(1, 784)
        with torch.no_grad():
            logps = model(img.to(device))

        ps = torch.exp(logps)
        probab = list(ps.cpu().numpy()[0])
        pred_label = probab.index(max(probab))
        true_label = labels.cpu().numpy()[i]
        if (true_label == pred_label):
            correct_count += 1
        all_count += 1

print("Broj testiranih slika =", all_count)
print("\nTočnost modela =", (correct_count / all_count))

# Example 4 - storage GPU

In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch import nn, optim
from PIL import Image

cuda = True if torch.cuda.is_available() else False
device = torch.device('cpu')
if cuda:
    device = torch.device('cuda')

transforms = torchvision.transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
trainset = datasets.MNIST('mnist_data', download=True, train=True, transform=transforms)
testset = datasets.MNIST('mnist_data', download=True, train=False, transform=transforms)


train_loader = DataLoader(trainset, batch_size=256, shuffle=True, num_workers=32, pin_memory=True)
test_loader = DataLoader(testset, batch_size=256, shuffle=True, num_workers=32, pin_memory=True)


input_size = 784
hidden_size_1 = 128
hidden_size_2 = 64
output_size = 10


model = nn.Sequential(nn.Linear(input_size, hidden_size_1),
                      nn.ReLU(),
                      nn.Linear(hidden_size_1, hidden_size_2),
                      nn.ReLU(),
                      nn.Linear(hidden_size_2, output_size),
                      nn.LogSoftmax(dim=1)).to(device)


loss_fn = nn.NLLLoss().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)


time0 = time()
epochs = 3
for e in range(epochs):
    running_loss = 0
    for images, labels in train_loader:

        images = images.to(device).view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model(images)
        loss = loss_fn(output, labels.to(device))
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    else:
        print("Epoch {} - Training loss: {}".format(e, running_loss / len(train_loader)))
print("\nTrenirali smo (u minutama):", (time() - time0) / 60)

torch.save(model, 'models/moj_model_gpu.pth')

loaded_model = torch.load('models/moj_model_gpu.pth')
print(model)


torch.save(model.state_dict(), 'models/moj_model_gpu_stateovi.pth')

model_load = nn.Sequential(nn.Linear(input_size, hidden_size_1),
                      nn.ReLU(),
                      nn.Linear(hidden_size_1, hidden_size_2),
                      nn.ReLU(),
                      nn.Linear(hidden_size_2, output_size),
                      nn.LogSoftmax(dim=1))

model_load.load_state_dict(torch.load('models/moj_model_gpu_stateovi.pth'))
model_load.to(device)

print(model_load)
# Dakle, pohrana je jako slična kada radimo s jednim GPU-om!
# U slučaju korištenja više GPU-ova, slučaj je malo drugačiji jer se koristi DataParallel model
# no to ćemo raditi na naprednim tehnikama treniranja.


# Example 5 - progress tqdm

In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time, sleep

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch import nn, optim
from PIL import Image
from tqdm import tqdm

cuda = True if torch.cuda.is_available() else False
device = torch.device('cpu')
if cuda:
    device = torch.device('cuda')

batch_size = 256

transforms = torchvision.transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
trainset = datasets.MNIST('mnist_data', download=True, train=True, transform=transforms)
testset = datasets.MNIST('mnist_data', download=True, train=False, transform=transforms)


train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=32, pin_memory=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=32, pin_memory=True)


input_size = 784
hidden_size_1 = 128
hidden_size_2 = 64
output_size = 10


model = nn.Sequential(nn.Linear(input_size, hidden_size_1),
                      nn.ReLU(),
                      nn.Linear(hidden_size_1, hidden_size_2),
                      nn.ReLU(),
                      nn.Linear(hidden_size_2, output_size),
                      nn.LogSoftmax(dim=1)).to(device)


loss_fn = nn.NLLLoss().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

epochs = 15
for e in range(epochs):
    # sjetimo se čemu služi enumerate!
    #Kreiramo petlju pomoću tqdm poziva. Kažemo mu ukupnu duljinu dataseta kako bi znao točno gdje smo.
    loop = tqdm(enumerate(train_loader), total=len(train_loader), leave=True)
    #Sada tu imamo i index zbog enumerate
    for idx, (images, labels) in loop:

        images = images.to(device).view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model(images)
        labels = labels.to(device)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()

        #Idemo odmah napraviti prikazivanje točnosti :)
        predictions = output.argmax(dim=1, keepdim=True).squeeze()
        correct = (predictions == labels).sum().item()
        accuracy = 100. * (correct / len(predictions))
        loop.set_description(f"Epoch [{e}/{epochs}")
        loop.set_postfix(loss = loss.item(), acc=accuracy)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to mnist_data/MNIST/raw



/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
Epoch [14/15: 100%|██████████| 235/235 [00:17<00:00, 13.76it/s, acc=96.9, loss=0.149]


# Example 6 - tensorboard

In [ ]:
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
from time import time, sleep

from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from torch import nn, optim
from PIL import Image

import pkbar
from torch.utils.tensorboard import SummaryWriter


cuda = True if torch.cuda.is_available() else False
device = torch.device('cpu')
if cuda:
    device = torch.device('cuda')

batch_size = 256
#Inicijalizacija writera
writer = SummaryWriter('runs/MNIST')

transforms = torchvision.transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])
trainset = datasets.MNIST('mnist_data', download=True, train=True, transform=transforms)
testset = datasets.MNIST('mnist_data', download=True, train=False, transform=transforms)


train_loader = DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=32, pin_memory=True)
test_loader = DataLoader(testset, batch_size=batch_size, shuffle=True, num_workers=32, pin_memory=True)


input_size = 784
hidden_size_1 = 128
hidden_size_2 = 64
output_size = 10


model = nn.Sequential(nn.Linear(input_size, hidden_size_1),
                      nn.ReLU(),
                      nn.Linear(hidden_size_1, hidden_size_2),
                      nn.ReLU(),
                      nn.Linear(hidden_size_2, output_size),
                      nn.LogSoftmax(dim=1)).to(device)


loss_fn = nn.NLLLoss().to(device)

optimizer = optim.SGD(model.parameters(), lr=0.003, momentum=0.9)

epochs = 15
train_per_epoch = int(len(trainset) / batch_size)
for e in range(epochs):
    kbar = pkbar.Kbar(target=train_per_epoch, epoch=e, num_epochs=epochs, width=20, always_stateful=False)
    for idx, (images, labels) in enumerate(train_loader):

        images = images.to(device).view(images.shape[0], -1)
        optimizer.zero_grad()
        output = model(images)
        labels = labels.to(device)
        loss = loss_fn(output, labels)
        loss.backward()
        optimizer.step()

        writer.add_scalar('loss', loss.item(), (e * train_per_epoch) + idx)
        predictions = output.argmax(dim=1, keepdim=True).squeeze()
        correct = (predictions == labels).sum().item()
        accuracy = correct / len(predictions)
        kbar.update(idx, values=[("loss", loss), ("acc", accuracy)])
        writer.add_scalar('acc', accuracy, (e * train_per_epoch) + idx)
        # Writer ima i funkcije za dodavanje ostalih tipova podataka. Slobodno ih istražite!

Epoch: 1/15


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 32 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


234/234 [====================] - 18s 76ms/step - loss: 1.3586 - acc: 0.6386
Epoch: 2/15
234/234 [====================] - 20s 84ms/step - loss: 0.4507 - acc: 0.8716
Epoch: 3/15
234/234 [====================] - 18s 77ms/step - loss: 0.3599 - acc: 0.8960
Epoch: 4/15
234/234 [====================] - 17s 73ms/step - loss: 0.3220 - acc: 0.9064
Epoch: 5/15
234/234 [====================] - 18s 76ms/step - loss: 0.2973 - acc: 0.9134
Epoch: 6/15
234/234 [====================] - 18s 78ms/step - loss: 0.2766 - acc: 0.9198
Epoch: 7/15
234/234 [====================] - 17s 74ms/step - loss: 0.2606 - acc: 0.9239
Epoch: 8/15
234/234 [====================] - 18s 77ms/step - loss: 0.2445 - acc: 0.9291
Epoch: 9/15
234/234 [====================] - 18s 77ms/step - loss: 0.2282 - acc: 0.9337
Epoch: 10/15
234/234 [====================] - 17s 74ms/step - loss: 0.2147 - acc: 0.9380
Epoch: 11/15
234/234 [====================] - 17s 73ms/step - loss: 0.2021 - acc: 0.9422
Epoch: 12/15
234/234 [====================


# LAB 4